# City and Weather API's

## City API

In [35]:
import pandas as pd
import os
local_project_path = r'C:\Users\antoi\\Documents\\Work&Learn\\JEDHA\\M03-DataCollection_Managment\\JEDHA-Projet-2-Kayak'
path = local_project_path+'\\my_booking\\my_booking'
print(path)
os.chdir(path)
chemin_fichier_csv = "./city.csv"
df = pd.read_csv(chemin_fichier_csv, header=None, names=['city','nan'])
cities = df['city'].str.replace(' ', '-').tolist()

# list(cities)

C:\Users\antoi\\Documents\\Work&Learn\\JEDHA\\M03-DataCollection_Managment\\JEDHA-Projet-2-Kayak\my_booking\my_booking


In [36]:
import requests

api_url = 'https://nominatim.openstreetmap.org/search'

enriched_cities = []

for city in cities:

    payload = {
        'city' : city,
        'limit' : 1,
        'country' : 'france',
        'country-code' : 'fr',
        'format' : 'json'
        }
    
    r_city = requests.get(api_url, 
                          params=payload)

    enriched_cities.append({'city': city,
                            'lat' : float(r_city.json()[0]['lat']),
                            'lon' : float(r_city.json()[0]['lon'])})
    
# display(enriched_cities)


In [37]:
df = pd.DataFrame.from_dict(enriched_cities)

In [38]:
import pandas as pd
import plotly.express as px
fig = px.scatter_mapbox(df,
                        lat='lat',
                        lon='lon',
                        center=dict(lat=46.2, lon=2.2),
                        zoom=4,
                        mapbox_style='carto-positron',
                        hover_name='city',
                        height=600)

fig.show()

## Weather API

In [39]:
os.chdir('../../')

In [41]:

with open("API_key.txt", "r") as file:
    API_key = str(file.readline())

exclude = 'current,minutely,hourly,alerts'
weather_cities = []

for city in enriched_cities:
    lat = city['lat']
    lon = city['lon']
    url = f'https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&exclude={exclude}&appid={API_key}&units=metric'
    weather_raw = requests.get(url).json()
    weather_raw['city']['name'] = city['city']
    weather_cities.append(weather_raw)

# display(weather_cities)

In [42]:
pd.json_normalize(weather_cities).head()

,cod,message,cnt,list,city.id,city.name,city.coord.lat,city.coord.lon,city.country,city.population,city.timezone,city.sunrise,city.sunset
0,200,0,40,"[{'dt': 1712059200, 'main': {'temp': 11.84, 'f...",6435453,Mont-Saint-Michel,48.6360,-1.5115,FR,0,7200,1712036461,1712083060
1,200,0,40,"[{'dt': 1712059200, 'main': {'temp': 12.1, 'fe...",2978640,St-Malo,48.6495,-2.0260,FR,50676,7200,1712036583,1712083184
2,200,0,40,"[{'dt': 1712059200, 'main': {'temp': 13.14, 'f...",3034483,Bayeux,49.2765,-0.7025,FR,15963,7200,1712036231,1712082902
3,200,0,40,"[{'dt': 1712059200, 'main': {'temp': 13.51, 'f...",3003796,Le-Havre,49.4939,0.1080,FR,185972,7200,1712036024,1712082719
4,200,0,40,"[{'dt': 1712059200, 'main': {'temp': 14.41, 'f...",6454707,Rouen,49.4405,1.0940,FR,108569,7200,1712035791,1712082479


In [43]:
pd.json_normalize(weather_cities)['city.name']
# pd.json_normalize(weather_cities)['list']
# pd.json_normalize(pd.json_normalize(weather_cities)['list'][34])
# pd.json_normalize(pd.json_normalize(weather_cities)['list'][34])['weather'][0]

0                Mont-Saint-Michel
1                          St-Malo
2                           Bayeux
3                         Le-Havre
4                            Rouen
5                            Paris
6                           Amiens
7                            Lille
8                       Strasbourg
9     Chateau-du-Haut-Koenigsbourg
10                          Colmar
11                       Eguisheim
12                        Besancon
13                           Dijon
14                          Annecy
15                        Grenoble
16                            Lyon
17                Gorges-du-Verdon
18              Bormes-les-Mimosas
19                          Cassis
20                       Marseille
21                 Aix-en-Provence
22                         Avignon
23                            Uzes
24                           Nimes
25                   Aigues-Mortes
26        Saintes-Maries-de-la-mer
27                       Collioure
28                  

In [44]:
weather_datas = pd.DataFrame()

for id, city in enumerate(pd.json_normalize(weather_cities)['city.name']):

    df3 = pd.json_normalize(pd.json_normalize(weather_cities)['list'][id])
    df3['city'] = city
    df3['lat'] = pd.json_normalize(weather_cities)["city.coord.lat"][id]
    df3['lon'] = pd.json_normalize(weather_cities)["city.coord.lon"][id]
    
    weather_datas = pd.concat([weather_datas, df3])

weather_datas['dt_txt'] = pd.to_datetime(weather_datas['dt_txt']) 
display(weather_datas.head(3))

,dt,weather,visibility,pop,dt_txt,main.temp,main.feels_like,main.temp_min,main.temp_max,main.pressure,...,main.temp_kf,clouds.all,wind.speed,wind.deg,wind.gust,rain.3h,sys.pod,city,lat,lon
0,1712059200,"[{'id': 500, 'main': 'Rain', 'description': 'l...",10000.0,1.0,2024-04-02 12:00:00,11.84,11.51,10.63,11.84,1006,...,1.21,83,6.61,195,12.40,1.64,d,Mont-Saint-Michel,48.636,-1.5115
1,1712070000,"[{'id': 500, 'main': 'Rain', 'description': 'l...",10000.0,1.0,2024-04-02 15:00:00,11.28,10.84,10.17,11.28,1006,...,1.11,89,9.21,181,16.81,2.15,d,Mont-Saint-Michel,48.636,-1.5115
2,1712080800,"[{'id': 500, 'main': 'Rain', 'description': 'l...",9925.0,1.0,2024-04-02 18:00:00,10.36,9.85,9.62,10.36,1006,...,0.74,94,9.31,181,17.48,2.00,d,Mont-Saint-Michel,48.636,-1.5115


In [45]:
col_drop = ['dt', 'weather', 'visibility', 'main.temp', 'main.temp_min', 'main.temp_max', 'main.pressure','main.sea_level',
            'main.grnd_level', 'main.humidity', 'main.temp_kf', 'clouds.all', 'wind.deg', 'wind.gust', 'rain.3h']

weather_datas.drop(columns=col_drop, inplace=True)


In [46]:
weather_during_day = weather_datas[weather_datas['sys.pod']=='d'].drop(columns='sys.pod')

In [47]:
weather_day_mean = weather_during_day.groupby('city').mean().reset_index()

In [48]:
display(weather_day_mean[['city','pop']].sort_values('pop').head())
display(weather_day_mean[['city','main.feels_like']].sort_values('main.feels_like',ascending=False).head())
display(weather_day_mean[['city','wind.speed']].sort_values('wind.speed').head())

,city,pop
1,Aix-en-Provence,0.0
30,Saintes-Maries-de-la-mer,0.0
24,Marseille,0.0
14,Collioure,0.0
11,Carcassonne,0.0


,city,main.feels_like
7,Bayonne,17.1988
4,Ariege,17.0876
14,Collioure,16.5784
9,Biarritz,16.5216
1,Aix-en-Provence,15.9540


,city,wind.speed
3,Annecy,2.0792
19,Grenoble,2.2256
4,Ariege,2.3264
7,Bayonne,2.5740
9,Biarritz,2.8328


Create a SCORE to select the city

In [49]:
weights = {'pop': -0.4, 'main.feels_like': 0.5, 'wind.speed': -0.1}

weather_day_mean['score'] = (weather_day_mean['pop'] * weights['pop'] +
                             weather_day_mean['main.feels_like'] * weights['main.feels_like'] +
                             weather_day_mean['wind.speed'] * weights['wind.speed'])

min_score = weather_day_mean['score'].min()
max_score = weather_day_mean['score'].max()
weather_day_mean['score'] = 10 * (weather_day_mean['score'] - min_score) / (max_score - min_score)

display(weather_day_mean.sort_values('score',ascending=False).head())

,city,pop,dt_txt,main.feels_like,wind.speed,lat,lon,score
7,Bayonne,0.0996,2024-04-04 21:36:00,17.1988,2.5740,43.4945,-1.4737,10.000000
4,Ariege,0.0440,2024-04-04 21:36:00,17.0876,2.3264,42.8458,1.6068,9.975072
9,Biarritz,0.0972,2024-04-04 21:36:00,16.5216,2.8328,43.4833,-1.5593,8.946294
14,Collioure,0.0000,2024-04-04 21:36:00,16.5784,3.9156,42.5251,3.0832,8.827451
5,Avignon,0.0136,2024-04-04 21:36:00,15.9116,3.0124,43.9492,4.8059,8.107087


In [50]:
fig = px.scatter_mapbox(weather_day_mean,
                        lat='lat',
                        lon='lon',
                        color='score',
                        size='score',
                        center=dict(lat=47, lon=2.2),
                        zoom=4.5,
                        color_continuous_scale=['white', 'yellow','orange', 'red','purple'],
                        mapbox_style='carto-positron',
                        hover_name='city',
                        hover_data=['main.feels_like', "pop", "wind.speed", "score"],
                        height=600,
                        width=600*16/9)

fig.show()

In [51]:
top5_cities = weather_day_mean.sort_values('score',ascending=False).head()
top5_cities['city']

7       Bayonne
4        Ariege
9      Biarritz
14    Collioure
5       Avignon
Name: city, dtype: object

In [52]:

fig = px.scatter_mapbox(top5_cities,
                        lat='lat',
                        lon='lon',
                        color='score',
                        size='score',
                        center=dict(lat=top5_cities['lat'].mean(), lon=top5_cities['lon'].mean()),
                        zoom=6,
                        color_continuous_scale=['white', 'yellow','orange', 'red','purple'],
                        mapbox_style='carto-positron',
                        text='city',
                        hover_name='city',
                        hover_data=['main.feels_like', "pop", "wind.speed", "score"],
                        height=600,
                        width=600*16/9)


fig.show()

In [53]:
weather_day_mean.rename(columns={
    'city': 'city_target',
    'pop': 'city_target_precipitation',
    'dt_txt': 'date_time',
    'main.feels_like': 'city_target_temp_feels_like',
    'wind.speed': 'city_target_wind_speed',
    'lat': 'lat_target',
    'lon': 'lon_target',
    'score': 'city_target_weather_score',
}, inplace=True)

In [54]:
weather_day_mean

,city_target,city_target_precipitation,date_time,city_target_temp_feels_like,city_target_wind_speed,lat_target,lon_target,city_target_weather_score
0,Aigues-Mortes,0.0096,2024-04-04 21:36:00,14.3072,4.9084,43.5662,4.1915,5.236875
1,Aix-en-Provence,0.0000,2024-04-04 21:36:00,15.9540,3.3804,43.5298,5.4475,8.077637
2,Amiens,0.4524,2024-04-04 21:36:00,13.4632,6.7316,49.8942,2.2957,2.971779
3,Annecy,0.2232,2024-04-04 21:36:00,12.7176,2.0792,45.8992,6.1289,3.505473
4,Ariege,0.0440,2024-04-04 21:36:00,17.0876,2.3264,42.8458,1.6068,9.975072
5,Avignon,0.0136,2024-04-04 21:36:00,15.9116,3.0124,43.9492,4.8059,8.107087
6,Bayeux,0.3480,2024-04-04 21:36:00,12.0492,7.4788,49.2765,-0.7025,0.826918
7,Bayonne,0.0996,2024-04-04 21:36:00,17.1988,2.5740,43.4945,-1.4737,10.000000
8,Besancon,0.2880,2024-04-04 21:36:00,13.4200,4.1732,47.2380,6.0244,3.841364
9,Biarritz,0.0972,2024-04-04 21:36:00,16.5216,2.8328,43.4833,-1.5593,8.946294


In [55]:
# weather_datas.to_csv('cities_weather_datas.csv',index=False)
weather_day_mean.to_csv('cities_weather_datas.csv',index=False)